In [9]:
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize
from math import sin, cos, sqrt, atan2, radians
import numpy as np


In [10]:
def flatten(data, col_list):
    for column in col_list:
        flattened = pd.DataFrame(dict(data[column])).transpose()
        columns = [str(col) for col in flattened.columns]
        flattened.columns = [column + '_' + colname for colname in columns]
        data = pd.concat([data, flattened], axis=1)
        data = data.drop(column, axis=1)
    return data

The chosen API contains information about the bike renting stations in London. The idea of the following exercise is that given a specific station, calculate the closest 5 stations and their occupancies.

The first step would be to show a list of stations by its name so the user can select the one s/he is at

In [11]:
response= requests.get("https://api-radon.tfl.gov.uk/BikePoint")
response.text
content=response.json()

In [12]:
cityBikes=pd.DataFrame(content)
stationNames= [e for e in cityBikes["commonName"]]
stationNames

['River Street , Clerkenwell',
 'Phillimore Gardens, Kensington',
 'Christopher Street, Liverpool Street',
 "St. Chad's Street, King's Cross",
 'Sedding Street, Sloane Square',
 'Broadcasting House, Marylebone',
 "Charlbert Street, St. John's Wood",
 'Maida Vale, Maida Vale',
 'New Globe Walk, Bankside',
 'Park Street, Bankside',
 'Brunswick Square, Bloomsbury',
 'Malet Street, Bloomsbury',
 'Scala Street, Fitzrovia',
 "Belgrove Street , King's Cross",
 'Great Russell Street, Bloomsbury',
 'Cartwright Gardens , Bloomsbury',
 'Hatton Wall, Holborn',
 'Drury Lane, Covent Garden',
 'Taviton Street, Bloomsbury',
 'Hampstead Road (Cartmel), Euston',
 'Northington Street , Holborn',
 'Red Lion Square, Holborn',
 'British Museum, Bloomsbury',
 'Doric Way , Somers Town',
 'Ampton Street , Clerkenwell',
 'Bouverie Street, Temple',
 'Bolsover Street, Fitzrovia',
 'Hereford Road, Bayswater',
 'Windsor Terrace, Hoxton',
 'Fanshaw Street, Hoxton',
 'Leonard Circus , Shoreditch',
 'Central House, Al

The user would have to give an input indicating the station name. For the purpose of this exercise we would assume that input is "River Street". Input by console to be implemented in the future

In [13]:
userInput="River Street"
"https://api.tfl.gov.uk/BikePoint/Search?query="+userInput

'https://api.tfl.gov.uk/BikePoint/Search?query=River Street'

In [14]:
response2= requests.get("https://api.tfl.gov.uk/BikePoint/Search?query="+userInput)
response2.text
content2=response2.json()

In [15]:
station=pd.DataFrame(content2)
station

,$type,additionalProperties,children,childrenUrls,commonName,id,lat,lon,placeType,url
0,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",[],[],[],"River Street , Clerkenwell",BikePoints_1,51.529163,-0.10997,BikePoint,/Place/BikePoints_1


We now take id of the station along with its latitude and longintude

In [17]:
stationId=station.loc[0,'id']
stationLat=station.loc[0,'lat']
stationLon=station.loc[0,'lon']

In [18]:
[stationId, stationLat, stationLon]

['BikePoints_1', 51.529163, -0.10997]

We are now ready to find the closest 5 stations

In [19]:
def calculateDistance(lati1,long1,lati2,long2):
    
    R = 6373.0

    lat1 = radians(lati1)
    lon1 = radians(long1)
    lat2 = radians(lati2)
    lon2 = radians(long2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    print(distance)
    

In [20]:
cityBikes.columns

Index(['$type', 'additionalProperties', 'children', 'childrenUrls',
       'commonName', 'id', 'lat', 'lon', 'placeType', 'url'],
      dtype='object')

In [21]:
coords=cityBikes [["id", 'lat','lon']]                    
coords.head()

,id,lat,lon
0,BikePoints_1,51.529163,-0.109970
1,BikePoints_2,51.499606,-0.197574
2,BikePoints_3,51.521283,-0.084605
3,BikePoints_4,51.530059,-0.120973
4,BikePoints_5,51.493130,-0.156876


In [22]:
dists=[]
for a in coords.itertuples():
    dist= calculateDistance(stationLat, stationLon, a[2], a[3])
    dists.append(dist)

0.0
6.897845858589301
1.962017312223106
0.7678718331081623
5.158211623819399
2.67031567492717
4.060846790785115
5.087698222088023
2.597079583728314
2.841263268487491
1.043162323729421
1.6424862160349767
2.0768109797680285
0.9482435689607719
1.7748023539047513
1.1509496585795278
0.8371087148362688
1.811530058900922
1.5361437490913668
2.000728262829078
0.8183654062513835
1.2538181050290211
1.51456273590971
1.5421713010158014
0.6129768675060475
1.7123358934553607
2.4113519555551353
6.0295358350055315
1.145156313331489
1.8423022073990234
1.835313957632805
3.1474789685861375
1.4523402827898042
6.021956159224909
7.071138796909054
6.999299342582009
2.2250132013890993
2.5368205259439534
2.04717115324705
1.1302199953722338
3.4141305984978354
3.130856506257373
3.6960673582698003
3.858521965011282
5.153198213867519
2.0008742686197567
3.4964738462629015
1.5586751150677491
0.4775752028504043
1.2488665445582134
3.151362668381152
1.3528759356581352
2.098736175209591
3.239354284881485
0.95917380424905

In [23]:
dists

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
coords["distance"]=np.array(dists)
coords
#Not sure why this does not work properly

In [ ]:
distDs=pd.Series(dists)
distDf=distDs.to_frame("distance")
dists


In [ ]:
occupancy for bike point: https://api.tfl.gov.uk/Occupancy/BikePoints/BikePoints_1

In [ ]:
response= requests.get("https://api-radon.tfl.gov.uk/BikePoint")

In [ ]:
for i in 